In [0]:
%run "../includes/configuration"

In [0]:
movie_df = spark.read.parquet(f"{silver_folder_path}/movies")

In [0]:
movie_df.display()

In [0]:
countries_df = spark.read.parquet(f"{silver_folder_path}/countries")
countries_df.display()

In [0]:
movie_df.display()

In [0]:
movies_companies_df = spark.read.parquet(f"{silver_folder_path}/movies_companies")

In [0]:
production_companies_df = spark.read.parquet(f"{silver_folder_path}/production_company")

In [0]:
productions_countries_df = spark.read.parquet(f"{silver_folder_path}/productions_countries")

In [0]:
productions_countries_df.display()

In [0]:
movies_companies_df =

In [0]:
movies_productions_df = movie_df.join(productions_countries_df, on='movie_id') \
    .select(movie_df['movie_id'], movie_df['title'], movie_df['budget'], 
            movie_df['revenue'], movie_df['duration_time'],  movie_df['year_release_date'], 
             movie_df['release_date'], productions_countries_df['country_id'] ) \
    .filter("year_release_date >= 2010")
movies_productions_df.display()

In [0]:
movies_productions_countries_df = movies_productions_df.join(countries_df, on='country_id') \
    .select(movies_productions_df['*'],  countries_df['country_name'])
movies_productions_countries_df.display()

In [0]:
production_companies_df.display()

In [0]:
movies_companies_df.display()

In [0]:
movies_productions_companies_df = movies_companies_df.join(production_companies_df, on='company_id', how="inner") \
    .select(movies_companies_df['company_id'], movies_companies_df['movie_id'], production_companies_df['company_name'])
display(movies_productions_companies_df)

In [0]:
movies_name_productions_countries_df = movies_productions_countries_df.join(movies_productions_companies_df, on = 'movie_id', how="inner")
movies_name_productions_countries_df.display()


In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
result_df = movies_name_productions_countries_df
result_final_df = result_df.orderBy("title", ascending=True) \
    .select( "title", "budget", "revenue", "duration_time",  "release_date", "country_name", "company_name") \
        .withColumn("created_date", current_timestamp()) 

result_final_df.display()

In [0]:
result_final_df.write.mode("overwrite").parquet(f"{gold_folder_path}/results_country_prod_company")

In [0]:
spark.read.parquet(f"{gold_folder_path}/results_country_prod_company").display()